#### Import all libraries

In [1]:
from __future__ import print_function
import os
import collections
import SimpleITK as sitk
import numpy as np
import six
import radiomics
from radiomics import firstorder, glcm, imageoperations, shape, glrlm, glszm, getTestCase
import scipy as sp
import matplotlib as mpl
import matplotlib.pyplot as plt
from pathlib import Path
import sys
import logging as logger
import time
import json
import os
import logging.config
import pandas as pd
import gc
import math

In [2]:
#Read the entire Benthiemer sandstone
global image_1, label_1
filename_ben = 'investigate_bent_8bit_1000cube.nrrd'
image_1 = sitk.ReadImage('D:\\Ankita\\pyradiomics\\data\\original_400_8bit.nrrd')

#### Extraction Settings

In [3]:
settings = {}
settings['label'] = 255
settings['binWidth'] = 2
#settings['binCount'] = 64

In [4]:
log_file = 'D:\\Ankita\\pyradiomics\\data\\log_file_m.txt'
handler = logging.FileHandler(filename=log_file, mode='w')  # overwrites log_files from previous runs. Change mode to 'a' to append.
formatter = logging.Formatter("%(levelname)s:%(name)s: %(message)s")  # format string for log messages
handler.setFormatter(formatter)
radiomics.logger.addHandler(handler)

# Control the amount of logging stored by setting the level of the logger. N.B. if the level is higher than the
# Verbositiy level, the logger level will also determine the amount of information printed to the output
radiomics.logger.setLevel(logging.DEBUG)

In [5]:
%%time
mask_x = 400
mask_y = 400
mask_z = 400

u_window_size = [400, 350, 300, 250, 200, 150, 100, 60, 50] #[1000, 900, 800, 700, 600, 500, 450, 400, 350, 300, 250, 200, 150, 100, 60, 50]
global window_size, total_iter, expect_iter, i_loc, j_loc, z_loc, bb, correctedMask, croppedImage, croppedMask, glcm_window
for window_size in u_window_size:     
    total_iter = 0
    my_entropy = []
    step_x = window_size-1
    step_y = window_size-1
    step_z = window_size-1
    
    i_loc=0
    j_loc=0
    z_loc=0
    i_loc_max = window_size
    j_loc_max = window_size
    z_loc_max = window_size
    
    while i_loc_max <= mask_x:
        #print("I loop", str(i_loc), str(i_loc_max), str(j_loc), str(j_loc_max), str(z_loc), str(z_loc_max))
        j_loc_max = window_size
        while j_loc_max <= mask_y:
            #print("J loop", str(i_loc), str(i_loc_max), str(j_loc), str(j_loc_max), str(z_loc), str(z_loc_max))
            z_loc_max = window_size
            while z_loc_max <= mask_z:
                #print("Z loop", str(i_loc), str(i_loc_max), str(j_loc), str(j_loc_max), str(z_loc), str(z_loc_max))
                full_mask = np.zeros((mask_x, mask_y, mask_z), dtype = np.uint8)
                full_mask[i_loc:i_loc_max,j_loc:j_loc_max,z_loc:z_loc_max] = 255
                #print(full_mask)
                
                label_1 = sitk.GetImageFromArray(full_mask, isVector=False)
                del full_mask

                bb, correctedMask = imageoperations.checkMask(image_1, label_1, label=255)
                if correctedMask is not None:
                    label_1 = correctedMask
                croppedImage, croppedMask = imageoperations.cropToTumorMask(image_1, label_1, bb)
                del correctedMask, label_1

                #calculate secondorder stats
                glcmFeatures = glcm.RadiomicsGLCM(croppedImage, croppedMask, **settings)
                glcmFeatures.enableFeatureByName('JointEntropy', True)
                result = glcmFeatures.execute()

                #print('Calculated GLCM features: ')
                for (key, val) in six.iteritems(result):
                    my_entropy.append(val)
                del key, val, result, glcmFeatures
                
                z_loc = z_loc + step_z
                total_iter = total_iter + 1
                print(total_iter)
                z_loc_max = z_loc+window_size
            
            z_loc = 0
            j_loc = j_loc + step_y
            j_loc_max = j_loc+window_size
        
        j_loc = 0
        i_loc = i_loc + step_x
        i_loc_max = i_loc+window_size
    
    my_df = pd.DataFrame(my_entropy)
    name_csv = 'ramstadentropy_window_' + str(window_size) + '.csv'
    my_df.to_csv(name_csv, index=False, header=False)
    print("Saved glcm values of window size ", str(window_size))
    del my_df, name_csv   
    print("Completed Calculations for Window ", str(window_size), ". The number of iterations is ", str(total_iter))

1
Saved glcm values of window size  400
Completed Calculations for Window  400 . The number of iterations is  1
1
Saved glcm values of window size  350
Completed Calculations for Window  350 . The number of iterations is  1
1
Saved glcm values of window size  300
Completed Calculations for Window  300 . The number of iterations is  1
1
Saved glcm values of window size  250
Completed Calculations for Window  250 . The number of iterations is  1
1
2
3
4
5
6
7
8
Saved glcm values of window size  200
Completed Calculations for Window  200 . The number of iterations is  8
1
2
3
4
5
6
7
8
Saved glcm values of window size  150
Completed Calculations for Window  150 . The number of iterations is  8
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
Saved glcm values of window size  100
Completed Calculations for Window  100 . The number of iterations is  64
1
2
3


In [ ]:
my_df = pd.DataFrame(my_entropy)
name_csv = 'simonentropy_window_' + str(window_size) + '.csv'
my_df.to_csv(name_csv, index=False, header=False)
print("Saved glcm values of window size ", str(window_size))
del my_df, name_csv   
print("Completed Calculations for Window ", str(window_size), ". The number of iterations is ", str(total_iter))

#### Only to be used for window sizes 600, 700, 800, 900

In [ ]:
%%time
mask_x = 1000
mask_y = 1000
mask_z = 1000

u_window_size = [1000, 900, 800, 700, 600, 700, 600] 
global window_size, total_iter, expect_iter, i_loc, j_loc, z_loc, bb, correctedMask, croppedImage, croppedMask, glcm_window
for window_size in u_window_size:     
    total_iter = 0
    my_entropy = []
    
    i_loc=0
    j_loc=0
    z_loc=0
    i_loc_max = window_size
    j_loc_max = window_size
    z_loc_max = window_size
    
    #print("Z loop", str(i_loc), str(i_loc_max), str(j_loc), str(j_loc_max), str(z_loc), str(z_loc_max))
    full_mask = np.zeros((mask_x, mask_y, mask_z), dtype = np.uint8)
    full_mask[i_loc:i_loc_max,j_loc:j_loc_max,z_loc:z_loc_max] = 255
    #print(full_mask)

    label_1 = sitk.GetImageFromArray(full_mask, isVector=False)
    del full_mask

    bb, correctedMask = imageoperations.checkMask(image_1, label_1, label=255)
    if correctedMask is not None:
        label_1 = correctedMask
    croppedImage, croppedMask = imageoperations.cropToTumorMask(image_1, label_1, bb)
    del correctedMask, label_1

    #calculate secondorder stats
    glcmFeatures = glcm.RadiomicsGLCM(croppedImage, croppedMask, **settings)
    glcmFeatures.enableFeatureByName('JointEntropy', True)
    result = glcmFeatures.execute()

    #print('Calculated GLCM features: ')
    for (key, val) in six.iteritems(result):
        my_entropy.append(val)
    del key, val, result, glcmFeatures
    
    my_df = pd.DataFrame(my_entropy)
    name_csv = 'entropy_window_' + str(window_size) + '.csv'
    my_df.to_csv(name_csv, index=False, header=False)
    print("Saved glcm values of window size ", str(window_size))
    del my_df, name_csv   
    print("Completed Calculations for Window ", str(window_size), ". The number of iterations is ", str(total_iter))